**Desription**:
    
10x

**NOTE:** CrowdVar Size Range

The following notebook is a revised version of the [Crowdvariant analysis notebook](https://github.com/lesleymaraina/NIST/blob/master/Notebooks/CrowdVariant_Analysis/CrowdVariantAnalysis%20-%20Machine%20Learning%5BAll%20Technologies%5D.ipynb). 

In the first part of this notebook, a random forest classifier was trained on crowdsourced[CrowdVariant](https://github.com/lesleymaraina/NIST/blob/master/Notebooks/CrowdVariant_Analysis/CrowdVariantAnalysis%20-%20Data%20Collection.ipynb) data for HG002 Deletions. 

In the second part of this notebook, the trained randfom forest classifier is then used to predict labels for a new set of randomly selected HG002 Deletions.

In [1]:
"""
Imports
"""
import pandas as pd
import numpy as np
from fancyimpute import KNN
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import LeaveOneOut
from scipy.stats import ks_2samp
from scipy import stats
from matplotlib import pyplot
from sklearn import preprocessing
from scipy.linalg import svd
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA as sklearnPCA
import plotly.plotly as py
from sklearn.cluster import DBSCAN
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import f1_score, precision_score
from sklearn import preprocessing
from ggplot import *
from bokeh.charts import TimeSeries
from bokeh.models import HoverTool
from bokeh.plotting import show
from bokeh.charts import Scatter, Histogram, output_file, show
from bokeh.plotting import figure, show, output_file, ColumnDataSource
from bokeh.io import output_notebook
from bokeh.charts import Bar, output_file, show
import bokeh.palettes as palettes
from bokeh.models import HoverTool, BoxSelectTool, Legend
from sklearn import (manifold, datasets, decomposition, ensemble,
                     discriminant_analysis, random_projection)

/Users/lmc2/anaconda/envs/NIHFAES/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning:

This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.

/Users/lmc2/anaconda/envs/NIHFAES/lib/python3.5/site-packages/sklearn/grid_search.py:43: DeprecationWarning:

This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.



***
Load Data
***
Restructure features so that the training dataset and prediction dataset have all of the same features

In [2]:
### Import Data
df_crowd = pd.read_csv('/Users/lmc2/NIST/Notebooks/CrowdVariant/tech_sep/crowd_var/CrowdVar_TenX.csv')
### Copy data in new dataframe as a later reference
df_crowd_2 = pd.read_csv('/Users/lmc2/NIST/Notebooks/CrowdVariant/tech_sep/crowd_var/CrowdVar_TenX.csv')

In [3]:
df_crowd.rename(columns={'size': 'Size'}, inplace=True)

In [4]:
df_crowd.head(3)

,chrom,start,end,sample,TenX.HP1_alt_alnScore_mean,TenX.HP1_alt_alnScore_std,TenX.HP1_alt_count,TenX.HP1_alt_insertSize_mean,TenX.HP1_alt_insertSize_std,TenX.HP1_alt_reason_alignmentScore,...,tandemrep_cnt,tandemrep_pct,segdup_cnt,segdup_pct,refN_cnt,refN_pct,Label,CN0_prob,CN1_prob,CN2_prob
0,1,187464828,187466479,HG002,0.0,0.000000,0.0,0.0,0.000000,0.0,...,8,0.096911,0,0.0,0,0,1,0.00,0.91,0.09
1,1,33156824,33157000,HG002,0.0,0.000000,0.0,0.0,0.000000,0.0,...,2,0.221591,0,0.0,0,0,1,0.04,0.91,0.05
2,1,53594099,53595428,HG002,528.0,16.573071,3.0,451.0,167.076829,2.0,...,3,0.059443,0,0.0,0,0,0,0.96,0.04,0.00


In [5]:
# Read in HG002 DEL dataframe
HG002_pred = pd.read_csv('/Users/lmc2/NIST/Notebooks/CrowdVariant/tech_sep/svviz/svviz_TenX.csv')
HG002_pred_2 = pd.read_csv('/Users/lmc2/NIST/Notebooks/CrowdVariant/tech_sep/svviz/svviz_TenX.csv')

In [6]:
# Store header names in lists and find names that are NOT contained in BOTH lists
c = list(df_crowd.columns.values)
d = list(HG002_pred.columns.values)
set(c) - set(d)

{'CN0_prob', 'CN1_prob', 'CN2_prob', 'Label'}

In [7]:
# a = ['chrom','start','end','sample','Ill300x.alt_alnScore_mean','Ill300x.alt_alnScore_std','Ill300x.alt_count','Ill300x.alt_insertSize_mean','Ill300x.alt_insertSize_std','Ill300x.alt_reason_alignmentScore','Ill300x.alt_reason_insertSizeScore','Ill300x.alt_reason_orientation','Ill300x.amb_alnScore_mean','Ill300x.amb_alnScore_std','Ill300x.amb_count','Ill300x.amb_insertSize_mean','Ill300x.amb_insertSize_std','Ill300x.amb_reason_alignmentScore_alignmentScore','Ill300x.amb_reason_alignmentScore_orientation','Ill300x.amb_reason_flanking','Ill300x.amb_reason_insertSizeScore_alignmentScore','Ill300x.amb_reason_insertSizeScore_insertSizeScore','Ill300x.amb_reason_multimapping','Ill300x.amb_reason_orientation_alignmentScore','Ill300x.amb_reason_orientation_orientation','Ill300x.amb_reason_same_scores','Ill300x.ref_alnScore_mean','Ill300x.ref_alnScore_std','Ill300x.ref_count','Ill300x.ref_insertSize_mean','Ill300x.ref_insertSize_std','Ill300x.ref_reason_alignmentScore','Ill300x.ref_reason_insertSizeScore','Ill300x.ref_reason_orientation','Ill300x.GT','Ill250.alt_alnScore_mean','Ill250.alt_alnScore_std','Ill250.alt_count','Ill250.alt_insertSize_mean','Ill250.alt_insertSize_std','Ill250.alt_reason_alignmentScore','Ill250.alt_reason_insertSizeScore','Ill250.alt_reason_orientation','Ill250.amb_alnScore_mean','Ill250.amb_alnScore_std','Ill250.amb_count','Ill250.amb_insertSize_mean','Ill250.amb_insertSize_std','Ill250.amb_reason_alignmentScore_alignmentScore','Ill250.amb_reason_alignmentScore_orientation','Ill250.amb_reason_flanking','Ill250.amb_reason_insertSizeScore_alignmentScore','Ill250.amb_reason_insertSizeScore_insertSizeScore','Ill250.amb_reason_multimapping','Ill250.amb_reason_orientation_alignmentScore','Ill250.amb_reason_orientation_orientation','Ill250.amb_reason_same_scores','Ill250.ref_alnScore_mean','Ill250.ref_alnScore_std','Ill250.ref_count','Ill250.ref_insertSize_mean','Ill250.ref_insertSize_std','Ill250.ref_reason_alignmentScore','Ill250.ref_reason_orientation','Ill250.GT','IllMP.alt_alnScore_mean','IllMP.alt_alnScore_std','IllMP.alt_count','IllMP.alt_insertSize_mean','IllMP.alt_insertSize_std','IllMP.alt_reason_alignmentScore','IllMP.alt_reason_insertSizeScore','IllMP.alt_reason_orientation','IllMP.amb_alnScore_mean','IllMP.amb_alnScore_std','IllMP.amb_count','IllMP.amb_insertSize_mean','IllMP.amb_insertSize_std','IllMP.amb_reason_alignmentScore_alignmentScore','IllMP.amb_reason_alignmentScore_orientation','IllMP.amb_reason_flanking','IllMP.amb_reason_insertSizeScore_insertSizeScore','IllMP.amb_reason_multimapping','IllMP.amb_reason_orientation_alignmentScore','IllMP.amb_reason_orientation_orientation','IllMP.amb_reason_same_scores','IllMP.ref_alnScore_mean','IllMP.ref_alnScore_std','IllMP.ref_count','IllMP.ref_insertSize_mean','IllMP.ref_insertSize_std','IllMP.ref_reason_alignmentScore','IllMP.ref_reason_insertSizeScore','IllMP.ref_reason_orientation','IllMP.GT','pacbio.alt_alnScore_mean','pacbio.alt_alnScore_std','pacbio.alt_count','pacbio.alt_insertSize_mean','pacbio.alt_insertSize_std','pacbio.alt_reason_alignmentScore','pacbio.amb_alnScore_mean','pacbio.amb_alnScore_std','pacbio.amb_count','pacbio.amb_insertSize_mean','pacbio.amb_insertSize_std','pacbio.amb_reason_alignmentScore_alignmentScore','pacbio.amb_reason_flanking','pacbio.amb_reason_multimapping','pacbio.amb_reason_same_scores','pacbio.ref_alnScore_mean','pacbio.ref_alnScore_std','pacbio.ref_count','pacbio.ref_insertSize_mean','pacbio.ref_insertSize_std','pacbio.ref_reason_alignmentScore','pacbio.GT','TenX.HP1_alt_alnScore_mean','TenX.HP1_alt_alnScore_std','TenX.HP1_alt_count','TenX.HP1_alt_insertSize_mean','TenX.HP1_alt_insertSize_std','TenX.HP1_alt_reason_alignmentScore','TenX.HP1_alt_reason_insertSizeScore','TenX.HP1_alt_reason_orientation','TenX.HP1_amb_alnScore_mean','TenX.HP1_amb_alnScore_std','TenX.HP1_amb_count','TenX.HP1_amb_insertSize_mean','TenX.HP1_amb_insertSize_std','TenX.HP1_amb_reason_alignmentScore_alignmentScore','TenX.HP1_amb_reason_alignmentScore_orientation','TenX.HP1_amb_reason_flanking','TenX.HP1_amb_reason_insertSizeScore_alignmentScore','TenX.HP1_amb_reason_multimapping','TenX.HP1_amb_reason_orientation_alignmentScore','TenX.HP1_amb_reason_orientation_orientation','TenX.HP1_amb_reason_same_scores','TenX.HP1_ref_alnScore_mean','TenX.HP1_ref_alnScore_std','TenX.HP1_ref_count','TenX.HP1_ref_insertSize_mean','TenX.HP1_ref_insertSize_std','TenX.HP1_ref_reason_alignmentScore','TenX.HP1_ref_reason_orientation','TenX.HP2_alt_alnScore_mean','TenX.HP2_alt_alnScore_std','TenX.HP2_alt_count','TenX.HP2_alt_insertSize_mean','TenX.HP2_alt_insertSize_std','TenX.HP2_alt_reason_alignmentScore','TenX.HP2_alt_reason_insertSizeScore','TenX.HP2_alt_reason_orientation','TenX.HP2_amb_alnScore_mean','TenX.HP2_amb_alnScore_std','TenX.HP2_amb_count','TenX.HP2_amb_insertSize_mean','TenX.HP2_amb_insertSize_std','TenX.HP2_amb_reason_alignmentScore_alignmentScore','TenX.HP2_amb_reason_alignmentScore_orientation','TenX.HP2_amb_reason_flanking','TenX.HP2_amb_reason_insertSizeScore_alignmentScore','TenX.HP2_amb_reason_multimapping','TenX.HP2_amb_reason_orientation_alignmentScore','TenX.HP2_amb_reason_orientation_orientation','TenX.HP2_amb_reason_same_scores','TenX.HP2_ref_alnScore_mean','TenX.HP2_ref_alnScore_std','TenX.HP2_ref_count','TenX.HP2_ref_insertSize_mean','TenX.HP2_ref_insertSize_std','TenX.HP2_ref_reason_alignmentScore','TenX.HP2_ref_reason_orientation','TenX.GT','size','GTcons','GTconflict','GTsupp','tandemrep_cnt','tandemrep_pct','segdup_cnt','segdup_pct','refN_cnt','refN_pct','Label','CN0_prob','CN1_prob','CN2_prob']
# b = ['chrom','start','end','sample','id','type','SVtype','Size','Ill300x.alt_alnScore_mean','Ill300x.alt_alnScore_std','Ill300x.alt_count','Ill300x.alt_insertSize_mean','Ill300x.alt_insertSize_std','Ill300x.alt_reason_alignmentScore','Ill300x.alt_reason_insertSizeScore','Ill300x.alt_reason_orientation','Ill300x.amb_alnScore_mean','Ill300x.amb_alnScore_std','Ill300x.amb_count','Ill300x.amb_insertSize_mean','Ill300x.amb_insertSize_std','Ill300x.amb_reason_alignmentScore_alignmentScore','Ill300x.amb_reason_alignmentScore_orientation','Ill300x.amb_reason_flanking','Ill300x.amb_reason_insertSizeScore_alignmentScore','Ill300x.amb_reason_insertSizeScore_insertSizeScore','Ill300x.amb_reason_insertSizeScore_orientation','Ill300x.amb_reason_multimapping','Ill300x.amb_reason_orientation_alignmentScore','Ill300x.amb_reason_orientation_orientation','Ill300x.amb_reason_same_scores','Ill300x.ref_alnScore_mean','Ill300x.ref_alnScore_std','Ill300x.ref_count','Ill300x.ref_insertSize_mean','Ill300x.ref_insertSize_std','Ill300x.ref_reason_alignmentScore','Ill300x.ref_reason_insertSizeScore','Ill300x.ref_reason_orientation','Ill300x.GT','Ill250.alt_alnScore_mean','Ill250.alt_alnScore_std','Ill250.alt_count','Ill250.alt_insertSize_mean','Ill250.alt_insertSize_std','Ill250.alt_reason_alignmentScore','Ill250.alt_reason_insertSizeScore','Ill250.alt_reason_orientation','Ill250.amb_alnScore_mean','Ill250.amb_alnScore_std','Ill250.amb_count','Ill250.amb_insertSize_mean','Ill250.amb_insertSize_std','Ill250.amb_reason_alignmentScore_alignmentScore','Ill250.amb_reason_alignmentScore_orientation','Ill250.amb_reason_flanking','Ill250.amb_reason_insertSizeScore_alignmentScore','Ill250.amb_reason_multimapping','Ill250.amb_reason_orientation_alignmentScore','Ill250.amb_reason_orientation_orientation','Ill250.amb_reason_same_scores','Ill250.ref_alnScore_mean','Ill250.ref_alnScore_std','Ill250.ref_count','Ill250.ref_insertSize_mean','Ill250.ref_insertSize_std','Ill250.ref_reason_alignmentScore','Ill250.ref_reason_orientation','Ill250.GT','IllMP.alt_alnScore_mean','IllMP.alt_alnScore_std','IllMP.alt_count','IllMP.alt_insertSize_mean','IllMP.alt_insertSize_std','IllMP.alt_reason_alignmentScore','IllMP.alt_reason_insertSizeScore','IllMP.alt_reason_orientation','IllMP.amb_alnScore_mean','IllMP.amb_alnScore_std','IllMP.amb_count','IllMP.amb_insertSize_mean','IllMP.amb_insertSize_std','IllMP.amb_reason_alignmentScore_alignmentScore','IllMP.amb_reason_alignmentScore_orientation','IllMP.amb_reason_flanking','IllMP.amb_reason_insertSizeScore_alignmentScore','IllMP.amb_reason_insertSizeScore_insertSizeScore','IllMP.amb_reason_multimapping','IllMP.amb_reason_orientation_alignmentScore','IllMP.amb_reason_orientation_orientation','IllMP.amb_reason_same_scores','IllMP.ref_alnScore_mean','IllMP.ref_alnScore_std','IllMP.ref_count','IllMP.ref_insertSize_mean','IllMP.ref_insertSize_std','IllMP.ref_reason_alignmentScore','IllMP.ref_reason_insertSizeScore','IllMP.ref_reason_orientation','IllMP.GT','TenX.HP1_alt_alnScore_mean','TenX.HP1_alt_alnScore_std','TenX.HP1_alt_count','TenX.HP1_alt_insertSize_mean','TenX.HP1_alt_insertSize_std','TenX.HP1_alt_reason_alignmentScore','TenX.HP1_alt_reason_insertSizeScore','TenX.HP1_alt_reason_orientation','TenX.HP1_amb_alnScore_mean','TenX.HP1_amb_alnScore_std','TenX.HP1_amb_count','TenX.HP1_amb_insertSize_mean','TenX.HP1_amb_insertSize_std','TenX.HP1_amb_reason_alignmentScore_alignmentScore','TenX.HP1_amb_reason_alignmentScore_orientation','TenX.HP1_amb_reason_flanking','TenX.HP1_amb_reason_insertSizeScore_alignmentScore','TenX.HP1_amb_reason_insertSizeScore_insertSizeScore','TenX.HP1_amb_reason_multimapping','TenX.HP1_amb_reason_orientation_alignmentScore','TenX.HP1_amb_reason_orientation_orientation','TenX.HP1_amb_reason_same_scores','TenX.HP1_ref_alnScore_mean','TenX.HP1_ref_alnScore_std','TenX.HP1_ref_count','TenX.HP1_ref_insertSize_mean','TenX.HP1_ref_insertSize_std','TenX.HP1_ref_reason_alignmentScore','TenX.HP1_ref_reason_insertSizeScore','TenX.HP1_ref_reason_orientation','TenX.HP2_alt_alnScore_mean','TenX.HP2_alt_alnScore_std','TenX.HP2_alt_count','TenX.HP2_alt_insertSize_mean','TenX.HP2_alt_insertSize_std','TenX.HP2_alt_reason_alignmentScore','TenX.HP2_alt_reason_insertSizeScore','TenX.HP2_alt_reason_orientation','TenX.HP2_amb_alnScore_mean','TenX.HP2_amb_alnScore_std','TenX.HP2_amb_count','TenX.HP2_amb_insertSize_mean','TenX.HP2_amb_insertSize_std','TenX.HP2_amb_reason_alignmentScore_alignmentScore','TenX.HP2_amb_reason_alignmentScore_orientation','TenX.HP2_amb_reason_flanking','TenX.HP2_amb_reason_insertSizeScore_alignmentScore','TenX.HP2_amb_reason_insertSizeScore_insertSizeScore','TenX.HP2_amb_reason_multimapping','TenX.HP2_amb_reason_orientation_alignmentScore','TenX.HP2_amb_reason_orientation_insertSizeScore','TenX.HP2_amb_reason_orientation_orientation','TenX.HP2_amb_reason_same_scores','TenX.HP2_ref_alnScore_mean','TenX.HP2_ref_alnScore_std','TenX.HP2_ref_count','TenX.HP2_ref_insertSize_mean','TenX.HP2_ref_insertSize_std','TenX.HP2_ref_reason_alignmentScore','TenX.HP2_ref_reason_orientation','TenX.GT','pacbio.alt_alnScore_mean','pacbio.alt_alnScore_std','pacbio.alt_count','pacbio.alt_insertSize_mean','pacbio.alt_insertSize_std','pacbio.alt_reason_alignmentScore','pacbio.amb_alnScore_mean','pacbio.amb_alnScore_std','pacbio.amb_count','pacbio.amb_insertSize_mean','pacbio.amb_insertSize_std','pacbio.amb_reason_alignmentScore_alignmentScore','pacbio.amb_reason_flanking','pacbio.amb_reason_multimapping','pacbio.amb_reason_same_scores','pacbio.ref_alnScore_mean','pacbio.ref_alnScore_std','pacbio.ref_count','pacbio.ref_insertSize_mean','pacbio.ref_insertSize_std','pacbio.ref_reason_alignmentScore','pacbio.GT','GTcons','GTconflict','GTsupp','tandemrep_cnt','tandemrep_pct','segdup_cnt','segdup_pct','refN_cnt','refN_pct','GIAB_Crowd']

# set(b) - set(a)

In [8]:
### Training Set: Drop irrelevant columns and GT information
df_crowd.drop(['GTcons'], axis=1, inplace = True)
df_crowd.drop(['GTconflict'], axis=1, inplace = True)
df_crowd.drop(['GTsupp'], axis=1, inplace = True)
# df_crowd.drop('SVtype', axis=1)
# df_crowd.drop(['type'],axis=1)
df_crowd.drop(['start'],axis=1, inplace = True)
df_crowd.drop(['end'],axis=1, inplace = True)
df_crowd.drop(['chrom'],axis=1, inplace = True)
# df_crowd.drop('Size',axis=1)
df_crowd.drop(['CN0_prob'],axis=1, inplace = True)
df_crowd.drop(['CN1_prob'],axis=1, inplace = True)
df_crowd.drop(['CN2_prob'],axis=1, inplace = True)
df_crowd.drop(['TenX.GT'],axis=1, inplace = True)
# df_crowd.drop(['pacbio.GT'],axis=1, inplace = True)
# df_crowd.drop(['IllMP.GT'],axis=1, inplace = True)
# df_crowd.drop(['Ill250.GT'],axis=1, inplace = True)
# df_crowd.drop(['Ill300x.GT'],axis=1, inplace = True)
df_crowd.drop(['sample'],axis=1, inplace = True)
# df_crowd.columns.drop(['Label'],axis=1, inplace = True)
# df_crowd.drop(['size'],axis=1, inplace = True)
# df_crowd.drop(['Svsize'],axis=1, inplace = True)
# df_crowd.drop(['Ill250.amb_reason_insertSizeScore_insertSizeScore'],axis=1, inplace = True)

In [9]:
# Store header names in lists and find names that are NOT contained in BOTH lists
c = list(df_crowd.columns.values)
d = list(HG002_pred.columns.values)
set(d) - set(c)

{'GIAB_Crowd',
 'GTconflict',
 'GTcons',
 'GTsupp',
 'SVtype',
 'TenX.GT',
 'TenX.HP1_amb_reason_insertSizeScore_insertSizeScore',
 'TenX.HP1_ref_reason_insertSizeScore',
 'TenX.HP2_amb_reason_insertSizeScore_insertSizeScore',
 'TenX.HP2_amb_reason_orientation_insertSizeScore',
 'chrom',
 'end',
 'id',
 'sample',
 'start',
 'type'}

In [10]:
### Prediction Set: Drop irrelevant columns and GT information
HG002_pred.drop(['GTcons'], axis=1, inplace = True)
HG002_pred.drop(['GTconflict'], axis=1, inplace = True)
HG002_pred.drop(['GTsupp'], axis=1, inplace = True)
HG002_pred.drop(['type'],axis=1, inplace = True)
HG002_pred.drop(['SVtype'],axis=1, inplace = True)
HG002_pred.drop(['start'],axis=1, inplace = True)
HG002_pred.drop(['end'],axis=1, inplace = True)
HG002_pred.drop(['chrom'],axis=1, inplace = True)
# HG002_pred.drop(['Size'],axis=1, inplace = True)
HG002_pred.drop(['TenX.GT'],axis=1, inplace = True)
# HG002_pred.drop(['pacbio.GT'],axis=1, inplace = True)
# HG002_pred.drop(['IllMP.GT'],axis=1, inplace = True)
# HG002_pred.drop(['Ill250.GT'],axis=1, inplace = True)
# HG002_pred.drop(['Ill300x.GT'],axis=1, inplace = True)
HG002_pred.drop(['sample'],axis=1, inplace = True)
HG002_pred.drop(['id'],axis=1, inplace = True)
# HG002_pred.drop(['Ill300x.amb_reason_insertSizeScore_orientation'],axis=1, inplace = True)
# HG002_pred.drop(['IllMP.amb_reason_insertSizeScore_alignmentScore'],axis=1, inplace = True)
HG002_pred.drop(['TenX.HP1_amb_reason_insertSizeScore_insertSizeScore'],axis=1, inplace = True)
HG002_pred.drop(['TenX.HP1_ref_reason_insertSizeScore'],axis=1, inplace = True)
HG002_pred.drop(['TenX.HP2_amb_reason_insertSizeScore_insertSizeScore'],axis=1, inplace = True)
HG002_pred.drop(['TenX.HP2_amb_reason_orientation_insertSizeScore'],axis=1, inplace = True)

In [11]:
df_crowd.head(3)

,TenX.HP1_alt_alnScore_mean,TenX.HP1_alt_alnScore_std,TenX.HP1_alt_count,TenX.HP1_alt_insertSize_mean,TenX.HP1_alt_insertSize_std,TenX.HP1_alt_reason_alignmentScore,TenX.HP1_alt_reason_insertSizeScore,TenX.HP1_alt_reason_orientation,TenX.HP1_amb_alnScore_mean,TenX.HP1_amb_alnScore_std,...,TenX.HP2_ref_reason_alignmentScore,TenX.HP2_ref_reason_orientation,Size,tandemrep_cnt,tandemrep_pct,segdup_cnt,segdup_pct,refN_cnt,refN_pct,Label
0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,500.708571,76.366265,...,2.0,0.0,1651,8,0.096911,0,0.0,0,0,1
1,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,484.000000,86.131843,...,12.0,0.0,176,2,0.221591,0,0.0,0,0,1
2,528.0,16.573071,3.0,451.0,167.076829,2.0,1.0,0.0,468.652174,89.142262,...,3.0,0.0,1329,3,0.059443,0,0.0,0,0,0


*** 
Impute missing values using KNN 
***

In [12]:
# Store data in a new variable which will be converted to a matrix
X = df_crowd
X.head(3)

,TenX.HP1_alt_alnScore_mean,TenX.HP1_alt_alnScore_std,TenX.HP1_alt_count,TenX.HP1_alt_insertSize_mean,TenX.HP1_alt_insertSize_std,TenX.HP1_alt_reason_alignmentScore,TenX.HP1_alt_reason_insertSizeScore,TenX.HP1_alt_reason_orientation,TenX.HP1_amb_alnScore_mean,TenX.HP1_amb_alnScore_std,...,TenX.HP2_ref_reason_alignmentScore,TenX.HP2_ref_reason_orientation,Size,tandemrep_cnt,tandemrep_pct,segdup_cnt,segdup_pct,refN_cnt,refN_pct,Label
0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,500.708571,76.366265,...,2.0,0.0,1651,8,0.096911,0,0.0,0,0,1
1,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,484.000000,86.131843,...,12.0,0.0,176,2,0.221591,0,0.0,0,0,1
2,528.0,16.573071,3.0,451.0,167.076829,2.0,1.0,0.0,468.652174,89.142262,...,3.0,0.0,1329,3,0.059443,0,0.0,0,0,0


In [13]:
# Convert dataframe to matrix
X=X.as_matrix()

#Imput missing values from three closest observations
X_imputed=KNN(k=3).complete(X)
X=pd.DataFrame(X_imputed)

Imputing row 1/1515 with 0 missing, elapsed time: 0.832
Imputing row 101/1515 with 0 missing, elapsed time: 0.879
Imputing row 201/1515 with 0 missing, elapsed time: 0.879
Imputing row 301/1515 with 0 missing, elapsed time: 0.879
Imputing row 401/1515 with 0 missing, elapsed time: 0.880
Imputing row 501/1515 with 0 missing, elapsed time: 0.880
Imputing row 601/1515 with 0 missing, elapsed time: 0.881
Imputing row 701/1515 with 0 missing, elapsed time: 0.881
Imputing row 801/1515 with 0 missing, elapsed time: 0.881
Imputing row 901/1515 with 0 missing, elapsed time: 0.882
Imputing row 1001/1515 with 0 missing, elapsed time: 0.882
Imputing row 1101/1515 with 0 missing, elapsed time: 0.883
Imputing row 1201/1515 with 0 missing, elapsed time: 0.883
Imputing row 1301/1515 with 0 missing, elapsed time: 0.883
Imputing row 1401/1515 with 0 missing, elapsed time: 0.884
Imputing row 1501/1515 with 0 missing, elapsed time: 0.884


In [14]:
# Store header values in a list, will be used later to re-label the matrix post KNN imputation
dfcrowd_header = list(df_crowd.columns.values)
X.columns = dfcrowd_header
X.head(3)

,TenX.HP1_alt_alnScore_mean,TenX.HP1_alt_alnScore_std,TenX.HP1_alt_count,TenX.HP1_alt_insertSize_mean,TenX.HP1_alt_insertSize_std,TenX.HP1_alt_reason_alignmentScore,TenX.HP1_alt_reason_insertSizeScore,TenX.HP1_alt_reason_orientation,TenX.HP1_amb_alnScore_mean,TenX.HP1_amb_alnScore_std,...,TenX.HP2_ref_reason_alignmentScore,TenX.HP2_ref_reason_orientation,Size,tandemrep_cnt,tandemrep_pct,segdup_cnt,segdup_pct,refN_cnt,refN_pct,Label
0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,500.708571,76.366265,...,2.0,0.0,1651.0,8.0,0.096911,0.0,0.0,0.0,0.0,1.0
1,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,484.000000,86.131843,...,12.0,0.0,176.0,2.0,0.221591,0.0,0.0,0.0,0.0,1.0
2,528.0,16.573071,3.0,451.0,167.076829,2.0,1.0,0.0,468.652174,89.142262,...,3.0,0.0,1329.0,3.0,0.059443,0.0,0.0,0.0,0.0,0.0


In [15]:
# Store Labels in a new 'Y' DataFrame
Y = pd.DataFrame()
Y['Label'] = X['Label']
#Y = X.pop('Label')

In [16]:
#Count the number of labels
pd.value_counts(Y['Label'].values, sort=False)

 1.0    959
 0.0    552
-1.0      1
 2.0      3
dtype: int64

In [17]:
# Remove labels from feature set
X.drop(['Label'],axis=1, inplace = True)

In [18]:
# Order features
X4 = X.reindex_axis(sorted(X.columns), axis=1)

***
Machine Learning
***

** Train RF Classifier **

In [19]:
# Train Test Split
# Train on 70% of the data and test on 30%
X_train, X_test, y_train, y_test = train_test_split(X4, Y, test_size=0.3, random_state=0)

In [20]:
# Train Random Forest Classifier
model = RandomForestClassifier() 
model.fit(X_train, y_train)

/Users/lmc2/anaconda/envs/NIHFAES/lib/python3.5/site-packages/ipykernel/__main__.py:3: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [21]:
#NOTE: Training Set - Show number of Hom Ref, Hom Var, Het Var datapoints the model was trained on
ytrain = pd.DataFrame()
ytrain['ytrain'] = y_train
pd.value_counts(ytrain['ytrain'].values, sort=False)

 1.0    671
 0.0     73
-1.0      1
 2.0      2
dtype: int64

** Precision Score **
- Overall model performance
- Using 30% of original dataset (test set)
- Truth labels: CrowdVariant labels

In [22]:
model.predict(X_test)

array([ 0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  0.,
        0.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  0.,  1.,
        0.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  0.,  0.,  1.,  0.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,
        0.,  1.,  1.,  1.,  0.,  0.,  1.,  1.,  1.,  1.,  0.,  0.,  1.,
        1.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  1.,  1.,  0.,  1.,  1.,
        1.,  1.,  0.,  1.,  1.,  0.,  1.,  0.,  1.,  0.,  1.,  1.,  0.,
        1.,  0.,  1.,  0.,  1.,  1.,  0.,  0.,  1.,  0.,  1.,  1.,  1.,
        1.,  0.,  1.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,
        0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  1.,  1.,
        0.,  0.,  1.,  1.,  0.,  1.,  1.,  1.,  0.,  0.,  1.,  0.,  0.,
        1.,  1.,  1.,  0.,  1.,  0.,  1.,  1.,  1.,  0.,  1.,  0.,  0.,
        1.,  0.,  0.,  1.,  1.,  0.,  0.,  1.,  0.,  0.,  1.,  1.,  0.,
        1.,  1.,  0.,  0.,  1.,  1.,  1.,  0.,  0.,  1.,  0.,  0

In [23]:
pred = model.predict(X_test)

In [24]:
precision_score(pred, y_test, average='micro') 

0.97142857142857142

In [25]:
# Add original labels and predicted labels back to the original dataframe
df_Xtest = pd.DataFrame(X_test)
df_Xtest.head()

,Size,TenX.HP1_alt_alnScore_mean,TenX.HP1_alt_alnScore_std,TenX.HP1_alt_count,TenX.HP1_alt_insertSize_mean,TenX.HP1_alt_insertSize_std,TenX.HP1_alt_reason_alignmentScore,TenX.HP1_alt_reason_insertSizeScore,TenX.HP1_alt_reason_orientation,TenX.HP1_amb_alnScore_mean,...,TenX.HP2_ref_insertSize_mean,TenX.HP2_ref_insertSize_std,TenX.HP2_ref_reason_alignmentScore,TenX.HP2_ref_reason_orientation,refN_cnt,refN_pct,segdup_cnt,segdup_pct,tandemrep_cnt,tandemrep_pct
1222,320.0,549.363636,11.145914,22.0,322.000000,104.180351,18.0,4.0,0.0,509.101124,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,1.0,0.12500
310,320.0,539.692308,15.982609,26.0,327.346154,77.329934,16.0,10.0,0.0,505.780000,...,351.789474,112.498926,38.0,0.0,0.0,0.0,0.0,0.000000,1.0,0.10625
9,233.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,503.578947,...,456.076923,157.563888,13.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000
785,121.0,538.571429,16.462284,21.0,336.285714,84.998679,21.0,0.0,0.0,503.455090,...,336.682927,104.134397,41.0,0.0,0.0,0.0,1.0,0.404959,1.0,0.31405
295,324.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,505.483051,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,2.0,0.16358


In [26]:
labels = pd.DataFrame(y_test)

In [27]:
df_crowd.head(3)

,TenX.HP1_alt_alnScore_mean,TenX.HP1_alt_alnScore_std,TenX.HP1_alt_count,TenX.HP1_alt_insertSize_mean,TenX.HP1_alt_insertSize_std,TenX.HP1_alt_reason_alignmentScore,TenX.HP1_alt_reason_insertSizeScore,TenX.HP1_alt_reason_orientation,TenX.HP1_amb_alnScore_mean,TenX.HP1_amb_alnScore_std,...,TenX.HP2_ref_reason_alignmentScore,TenX.HP2_ref_reason_orientation,Size,tandemrep_cnt,tandemrep_pct,segdup_cnt,segdup_pct,refN_cnt,refN_pct,Label
0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,500.708571,76.366265,...,2.0,0.0,1651,8,0.096911,0,0.0,0,0,1
1,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,484.000000,86.131843,...,12.0,0.0,176,2,0.221591,0,0.0,0,0,1
2,528.0,16.573071,3.0,451.0,167.076829,2.0,1.0,0.0,468.652174,89.142262,...,3.0,0.0,1329,3,0.059443,0,0.0,0,0,0


In [28]:
df_Xtest['predicted_label'] = pred
df_Xtest['Label'] = df_crowd_2['Label']
df_Xtest['chrom'] = df_crowd_2['chrom']
df_Xtest['start'] = df_crowd_2['start']
df_Xtest['end'] = df_crowd_2['end']
# df_Xtest['Y_test'] = labels

In [29]:
pd.value_counts(df_Xtest['Label'].values, sort=False)

0    166
1    288
2      1
dtype: int64

In [30]:
pd.value_counts(df_Xtest['predicted_label'].values, sort=False)

0.0    172
1.0    283
dtype: int64

In [31]:
from sklearn.metrics import confusion_matrix
ytest = df_Xtest['Label']
predict = df_Xtest['predicted_label']
print(confusion_matrix(ytest, predict))

[[163   3   0]
 [  9 279   0]
 [  0   1   0]]


In [32]:
pd.crosstab(ytest, predict, rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,0.0,1.0,All
True,,,
0,163,3,166
1,9,279,288
2,0,1,1
All,172,283,455


In [33]:
from sklearn.metrics import classification_report
print(classification_report(ytest, predict))

             precision    recall  f1-score   support

          0       0.95      0.98      0.96       166
          1       0.99      0.97      0.98       288
          2       0.00      0.00      0.00         1

avg / total       0.97      0.97      0.97       455



/Users/lmc2/anaconda/envs/NIHFAES/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.



***
Predict
***

**Load Data**

In [34]:
HG002_pred.head(3)

,Size,TenX.HP1_alt_alnScore_mean,TenX.HP1_alt_alnScore_std,TenX.HP1_alt_count,TenX.HP1_alt_insertSize_mean,TenX.HP1_alt_insertSize_std,TenX.HP1_alt_reason_alignmentScore,TenX.HP1_alt_reason_insertSizeScore,TenX.HP1_alt_reason_orientation,TenX.HP1_amb_alnScore_mean,...,TenX.HP2_ref_insertSize_std,TenX.HP2_ref_reason_alignmentScore,TenX.HP2_ref_reason_orientation,tandemrep_cnt,tandemrep_pct,segdup_cnt,segdup_pct,refN_cnt,refN_pct,GIAB_Crowd
0,-178,534.0,0.0,1.0,712.0,0.0,0.0,1.0,0.0,501.088000,...,94.881452,10.0,0.0,0,0.0,1,0.679775,0,0,2
1,-90,558.0,0.0,1.0,1027.0,0.0,0.0,1.0,0.0,503.252000,...,61.867241,6.0,0.0,0,0.0,1,1.000000,0,0,2
2,-33,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,509.014706,...,74.279203,30.0,0.0,0,0.0,0,0.000000,0,0,2


In [35]:
X2 = HG002_pred

** Impute missing values using KNN **

In [36]:
#Convert dataframe to matrix
X2=X2.as_matrix()

#Imput missing values from three closest observations
X2_imputed=KNN(k=3).complete(X2)
X2=pd.DataFrame(X2_imputed)

Imputing row 1/2805 with 0 missing, elapsed time: 2.849
Imputing row 101/2805 with 0 missing, elapsed time: 2.863
Imputing row 201/2805 with 0 missing, elapsed time: 2.870
Imputing row 301/2805 with 0 missing, elapsed time: 2.876
Imputing row 401/2805 with 0 missing, elapsed time: 2.881
Imputing row 501/2805 with 0 missing, elapsed time: 2.883
Imputing row 601/2805 with 0 missing, elapsed time: 2.887
Imputing row 701/2805 with 0 missing, elapsed time: 2.891
Imputing row 801/2805 with 0 missing, elapsed time: 2.893
Imputing row 901/2805 with 0 missing, elapsed time: 2.897
Imputing row 1001/2805 with 0 missing, elapsed time: 2.897
Imputing row 1101/2805 with 0 missing, elapsed time: 2.898
Imputing row 1201/2805 with 0 missing, elapsed time: 2.900
Imputing row 1301/2805 with 0 missing, elapsed time: 2.904
Imputing row 1401/2805 with 0 missing, elapsed time: 2.908
Imputing row 1501/2805 with 0 missing, elapsed time: 2.908
Imputing row 1601/2805 with 0 missing, elapsed time: 2.910
Imputing 

In [37]:
dfcrowd_header = list(HG002_pred.columns.values)
X2.columns = dfcrowd_header
X2.head(3)

,Size,TenX.HP1_alt_alnScore_mean,TenX.HP1_alt_alnScore_std,TenX.HP1_alt_count,TenX.HP1_alt_insertSize_mean,TenX.HP1_alt_insertSize_std,TenX.HP1_alt_reason_alignmentScore,TenX.HP1_alt_reason_insertSizeScore,TenX.HP1_alt_reason_orientation,TenX.HP1_amb_alnScore_mean,...,TenX.HP2_ref_insertSize_std,TenX.HP2_ref_reason_alignmentScore,TenX.HP2_ref_reason_orientation,tandemrep_cnt,tandemrep_pct,segdup_cnt,segdup_pct,refN_cnt,refN_pct,GIAB_Crowd
0,-178.0,534.0,0.0,1.0,712.0,0.0,0.0,1.0,0.0,501.088000,...,94.881452,10.0,0.0,0.0,0.0,1.0,0.679775,0.0,0.0,2.0
1,-90.0,558.0,0.0,1.0,1027.0,0.0,0.0,1.0,0.0,503.252000,...,61.867241,6.0,0.0,0.0,0.0,1.0,1.000000,0.0,0.0,2.0
2,-33.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,509.014706,...,74.279203,30.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,2.0


In [38]:
X3 = pd.DataFrame()
X3 = X2
X3.head(3)

,Size,TenX.HP1_alt_alnScore_mean,TenX.HP1_alt_alnScore_std,TenX.HP1_alt_count,TenX.HP1_alt_insertSize_mean,TenX.HP1_alt_insertSize_std,TenX.HP1_alt_reason_alignmentScore,TenX.HP1_alt_reason_insertSizeScore,TenX.HP1_alt_reason_orientation,TenX.HP1_amb_alnScore_mean,...,TenX.HP2_ref_insertSize_std,TenX.HP2_ref_reason_alignmentScore,TenX.HP2_ref_reason_orientation,tandemrep_cnt,tandemrep_pct,segdup_cnt,segdup_pct,refN_cnt,refN_pct,GIAB_Crowd
0,-178.0,534.0,0.0,1.0,712.0,0.0,0.0,1.0,0.0,501.088000,...,94.881452,10.0,0.0,0.0,0.0,1.0,0.679775,0.0,0.0,2.0
1,-90.0,558.0,0.0,1.0,1027.0,0.0,0.0,1.0,0.0,503.252000,...,61.867241,6.0,0.0,0.0,0.0,1.0,1.000000,0.0,0.0,2.0
2,-33.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,509.014706,...,74.279203,30.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,2.0


In [39]:
X2.drop(['GIAB_Crowd'],axis=1, inplace = True)

In [40]:
# Order features
X5 = X2.reindex_axis(sorted(X2.columns), axis=1)

In [41]:
model.predict(X5)

array([ 0.,  1.,  1., ...,  0.,  0.,  0.])

In [42]:
df_new = pd.DataFrame()
df_new['true'] = HG002_pred['GIAB_Crowd'].astype(float)
df_new['predicted'] = model.predict(X5)
df_new['predicted'] = df_new['predicted'].astype(float)

In [43]:
pd.value_counts(df_new['true'].values, sort=False)

2.0     473
1.0    1734
0.0     598
dtype: int64

In [44]:
pd.value_counts(df_new['predicted'].values, sort=False)

0.0     876
1.0    1927
2.0       2
dtype: int64

In [45]:
# Step for processing missing values in label column
df_new_ = pd.DataFrame()
df_new_ = df_new[df_new['true'].notnull()]
df_new_

,true,predicted
0,2.0,0.0
1,2.0,1.0
2,2.0,1.0
3,2.0,1.0
4,2.0,1.0
5,2.0,1.0
6,2.0,1.0
7,2.0,1.0
8,2.0,1.0
9,2.0,1.0


In [ ]:
df_new['true']

In [46]:
# Calculate Precision Score
true = df_new_['true']
predicted = df_new_['predicted']
precision_score(true, predicted, average='micro') 

0.73368983957219247

In [47]:
pd.value_counts(df_new['true'].values, sort=False)

2.0     473
1.0    1734
0.0     598
dtype: int64

In [48]:
pd.crosstab(true, predicted, rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,0.0,1.0,2.0,All
True,,,,
0.0,574,24,0,598
1.0,248,1484,2,1734
2.0,54,419,0,473
All,876,1927,2,2805


In [49]:
from sklearn.metrics import classification_report
true = df_new_['true']
pred = df_new_['predicted']
print(classification_report(true, pred))

             precision    recall  f1-score   support

        0.0       0.66      0.96      0.78       598
        1.0       0.77      0.86      0.81      1734
        2.0       0.00      0.00      0.00       473

avg / total       0.62      0.73      0.67      2805



In [50]:
# # Store Chr X and Y as numbers (23, 24)
# X5['chrom'].replace('X', 23, inplace=True)
# X5['chrom'].replace('Y', 24, inplace=True)

In [51]:
pred_prob = model.predict_proba(X5)

In [52]:
X5['predicted_label'] = pred
X5['Label'] = HG002_pred_2['GIAB_Crowd']
X5['chrom'] = HG002_pred_2['chrom']
X5['start'] = HG002_pred_2['start']
X5['end'] = HG002_pred_2['end']

In [53]:
X6 = pd.concat([X5, pd.DataFrame(pred_prob, columns=['1','2','3','4'])])

In [57]:
X6.to_csv('10X_pred_prob.csv', index=False)

In [58]:
X6 = pd.read_csv('10X_pred_prob.csv')

In [59]:
X6.rename(columns={'1': 'unknown'}, inplace=True)
X6.rename(columns={'2': 'Hom_Var'}, inplace=True)
X6.rename(columns={'3': 'Het_Var'}, inplace=True)
X6.rename(columns={'4': 'Hom_Ref'}, inplace=True)

** Show Size Distribution of Labels with Predict Probability > 0.8 **

In [60]:
X6['Hom_Var_str'] = X6['Hom_Var'].astype(str)

In [61]:
X_homVar_conf = X6[(X6['Hom_Var'] == 0.9) | (X6['Hom_Var'] == 1) | (X6['Hom_Var'] == 0.8)]

In [62]:
X_homVar_conf['Size'] = HG002_pred_2['Size'].abs()

/Users/lmc2/anaconda/envs/NIHFAES/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [63]:
output_notebook()

Loading BokehJS ...

** Homozygous **

In [64]:
# all_df_del['abs_size'] = all_df_del['Size'].abs()
X_homVar_conf['Size_Log'] = np.log10(X_homVar_conf.Size)

/Users/lmc2/anaconda/envs/NIHFAES/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [65]:
# Plot the counts of each predicted probability for Het Var events
p = figure()
p = Histogram(X_homVar_conf, values='Size_Log', title='Homozygous Variant: Predict Proba > 0.8', color='LightSlateGray', bins=30, xlabel="Size(Log10)", ylabel="Frequency")
output_file("10X_HomVar_PredProb0.8+.html")
show(p)

![Figure 2](https://raw.githubusercontent.com/lesleymaraina/NIST/master/Notebooks/CrowdVariant_Analysis/Images/10X.hom.0.8%2B.png)

** Heterozygous **

In [66]:
X6['Het_Var_str'] = X6['Het_Var'].astype(str)
X_hetVar_conf = X6[(X6['Het_Var'] == 0.9) | (X6['Het_Var'] == 1) | (X6['Het_Var'] == 0.8)]
X_hetVar_conf['Size'] = HG002_pred_2['Size'].abs()
X_hetVar_conf['Size_Log'] = np.log10(X_hetVar_conf.Size)

/Users/lmc2/anaconda/envs/NIHFAES/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/lmc2/anaconda/envs/NIHFAES/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [68]:
# Plot the counts of each predicted probability for Het Var events
p = figure()
p = Histogram(X_hetVar_conf, values='Size_Log', title='Heterozygous Variant: Predict Proba > 0.8', color='LightSlateGray', bins=30, xlabel="Size(log10)", ylabel="Frequency")
output_file("10X_HetVar_PredProb0.8+.html")
show(p)

![Figure 2](https://raw.githubusercontent.com/lesleymaraina/NIST/master/Notebooks/CrowdVariant_Analysis/Images/10X.het.0.8%2B.png)

** Show Size Distribution of Labels with Predict Probability < 0.3 **

In [69]:
X6['Hom_Var_str'] = X6['Hom_Var'].astype(str)
X_homVar_least = X6[(X6['Hom_Var'] == 0.3) | (X6['Hom_Var'] == 0.2) | (X6['Hom_Var'] == 0.1)]
X_homVar_least['Size'] = HG002_pred_2['Size'].abs()
X_homVar_least['Size_Log'] = np.log10(X_homVar_least.Size)

/Users/lmc2/anaconda/envs/NIHFAES/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/lmc2/anaconda/envs/NIHFAES/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [70]:
# Plot the counts of each predicted probability for Het Var events
p = figure()
p = Histogram(X_homVar_least, values='Size_Log', title='Homozygous Variant: Predict Proba < 0.3', color='LightSlateGray', bins=30, xlabel="Size(log10)", ylabel="Frequency")
output_file("300x_HomVar_PredProb0.3-.html")
show(p)

INFO:bokeh.core.state:Session output file '300x_HomVar_PredProb0.3-.html' already exists, will be overwritten.


![Figure 1](https://raw.githubusercontent.com/lesleymaraina/NIST/master/Notebooks/CrowdVariant_Analysis/Images/10X.hom.0.3-.png)

** Heterozygous **

In [71]:
X6['Het_Var_str'] = X6['Het_Var'].astype(str)
X_hetVar_least = X6[(X6['Het_Var'] == 0.3) | (X6['Het_Var'] == 0.2) | (X6['Het_Var'] == 0.1)]
X_hetVar_least['Size'] = HG002_pred_2['Size'].abs()
X_hetVar_least['Size_Log'] = np.log10(X_hetVar_least.Size)

/Users/lmc2/anaconda/envs/NIHFAES/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/lmc2/anaconda/envs/NIHFAES/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [72]:
# Plot the counts of each predicted probability for Het Var events
p = figure()
p = Histogram(X_hetVar_least, values='Size_Log', title='Heterozygous Variant: Predict Proba < 0.3', color='LightSlateGray', bins=30, xlabel="Size(log10)", ylabel="Frequency")
output_file("10x_HetVar_PredProb0.3-.html")
show(p)

INFO:bokeh.core.state:Session output file '300x_HetVar_PredProb0.3-.html' already exists, will be overwritten.


![Figure1](https://raw.githubusercontent.com/lesleymaraina/NIST/master/Notebooks/CrowdVariant_Analysis/Images/10X.het.0.3-.png)

** Size Distribution of Original Data **

In [ ]:
p = figure()
p = Histogram(df_crowd_2, values='size', title='CrowdVariant Data Size Distribution', color='LightSlateGray', bins=30, xlabel="Size(bp)", ylabel="Frequency")
output_file("CrowdVar.Size.html")
show(p)

![Figure1](https://raw.githubusercontent.com/lesleymaraina/NIST/master/Notebooks/crowdVar_size.png)

In [ ]:
del_data = pd.read_csv('/Users/lmc2/Desktop/log_Size_.csv')

In [ ]:
p = figure()
p = Histogram(del_data, values='DEL_log_size', title='SVVIZ HG002 Rand Selection Data Size Distribution', color='LightSlateGray', bins=30, xlabel="Size(log10)", ylabel="Frequency")
output_file("SVVIZ.Size.html")
show(p)

![Figure1](https://raw.githubusercontent.com/lesleymaraina/NIST/master/Notebooks/svviz_size.png)

**TODO**

[Feature Importance](http://scikit-learn.org/stable/auto_examples/ensemble/plot_forest_importances.html)